Un notebook propre et commenté (introduction,
analyses, conclusion, etc) contenant : le procédé de
développement de votre outil, du prétraitement à la
modélisation des données, en passant par l’analyse
exploratoire.

Pensez à répondre à la problématique.

In [41]:
import pandas as pd 
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [42]:
df = pd.read_csv('data\\raw\\marketing_campaign.csv' , sep='	')

df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2240 entries, 0 to 2239
Data columns (total 29 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   2240 non-null   int64  
 1   Year_Birth           2240 non-null   int64  
 2   Education            2240 non-null   object 
 3   Marital_Status       2240 non-null   object 
 4   Income               2216 non-null   float64
 5   Kidhome              2240 non-null   int64  
 6   Teenhome             2240 non-null   int64  
 7   Dt_Customer          2240 non-null   object 
 8   Recency              2240 non-null   int64  
 9   MntWines             2240 non-null   int64  
 10  MntFruits            2240 non-null   int64  
 11  MntMeatProducts      2240 non-null   int64  
 12  MntFishProducts      2240 non-null   int64  
 13  MntSweetProducts     2240 non-null   int64  
 14  MntGoldProds         2240 non-null   int64  
 15  NumDealsPurchases    2240 non-null   i

In [43]:
# on voit qu'il y a 24 null dans la colonne Income , nous allons donc les supprimer pour ne pas fausser le model
print(df.isnull().sum())

df = df.dropna(subset=['Income'])

ID                      0
Year_Birth              0
Education               0
Marital_Status          0
Income                 24
Kidhome                 0
Teenhome                0
Dt_Customer             0
Recency                 0
MntWines                0
MntFruits               0
MntMeatProducts         0
MntFishProducts         0
MntSweetProducts        0
MntGoldProds            0
NumDealsPurchases       0
NumWebPurchases         0
NumCatalogPurchases     0
NumStorePurchases       0
NumWebVisitsMonth       0
AcceptedCmp3            0
AcceptedCmp4            0
AcceptedCmp5            0
AcceptedCmp1            0
AcceptedCmp2            0
Complain                0
Z_CostContact           0
Z_Revenue               0
Response                0
dtype: int64


In [44]:
# certaines données sont erronées pour les années de naissance , au lieu d'etre a 1900 c'est a 1800 
print(df[df['Year_Birth'] < 1900]['Year_Birth'])

239    1893
339    1899
Name: Year_Birth, dtype: int64


In [45]:
#passage de la colonne Dt_Customer en datetime
df['Dt_Customer'] = pd.to_datetime(df['Dt_Customer'], format="%d-%m-%Y", errors='coerce')
# toutes les dt_customers sont situés entre 2012 et 2015
df[(df['Dt_Customer'] > '01-01-2015') | (df['Dt_Customer'] < '01-01-2012')]

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response


In [46]:
# Séparation des valeurs numériques et des strings
df_num = df.select_dtypes(include=['int64', 'float64'])
df_str = df.select_dtypes(include=['object'])

# Standardiser et PCA sur numériques car on ne peux pas le faire sur des Chaines de caractères
scaler = StandardScaler()
df_num_scaled = scaler.fit_transform(df_num)

pca = PCA(n_components=5)
df_pca = pca.fit_transform(df_num_scaled)

# Transformer en dataframe avec noms de colonnes
df_num_pca_and_scaled = pd.DataFrame(df_pca, columns=[f'PC{i+1}' for i in range(df_pca.shape[1])])

# Recomposer dataframe final
marketing_campaign_clean = pd.concat([df_num_pca_and_scaled, df_str.reset_index(drop=True)], axis=1)
marketing_campaign_clean.to_csv('data\\processed\\marketing_campaign_clean.csv', index=False, sep=',')